In [1]:
from __future__ import annotations
from math import sqrt, pi, atan2, cos, sin


class Complex:
    x: float = 0  # real
    y: float = 0  # imaginary

    def __init__(self):
        self.x = 0
        self.y = 0

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other: Complex) -> Complex:
        ret = Complex(0, 0)
        ret.x = self.x + other.x
        ret.y = self.y + other.y
        return ret

    def __sub__(self, other: Complex) -> Complex:
        ret = Complex(0, 0)
        ret.x = self.x - other.x
        ret.y = self.y - other.y
        return ret

    def __mul__(self, other: Complex) -> Complex:
        ret = Complex(0, 0)
        ret.x = self.x * other.x - self.y * other.y
        ret.y = self.x * other.y + self.y * other.x
        return ret

    def __truediv__(self, other: Complex) -> Complex:
        ret = Complex(0, 0)
        ret.x = (self.x * other.x + self.y * other.y) / (
            other.x * other.x + other.y * other.y
        )
        ret.y = (self.y * other.x - self.x * other.y) / (
            other.x * other.x + other.y * other.y
        )
        return ret

    def __str__(self):
        if self.y == 0:
            return round(self.x, 2).__str__() + "\t"
        elif self.x == 0:
            return round(self.y, 2).__str__() + "i\t"
        else:
            return (
                round(self.x, 2).__str__() + " + " + round(self.y, 2).__str__() + "i"
            )

    def __eq__(self, other: Complex) -> bool:
        return self.x == other.x and self.y == other.y

    def modulus(self) -> float:
        return sqrt(self.x * self.x + self.y * self.y)

    def conjugate(self) -> Complex:
        return Complex(self.x, -self.y)

    def negate(self) -> Complex:
        return Complex(-self.x, -self.y)

    def toPolar(self) -> Polar:
        r = self.modulus()
        theta = atan2(self.y, self.x)
        return Polar(r, theta)

    def pow(self, n: int) -> Complex:
        polar = self.toPolar()
        polar.r = polar.r**n
        polar.theta = polar.theta * n
        return polar.toCartesian()

    def root(self, n: int) -> [Complex]:
        ret = []
        polar = self.toPolar()
        polar.r = polar.r ** (1 / n)
        for i in range(n):
            polar.theta = (polar.theta + 2 * pi * i) / n
            ret.append(polar.toCartesian())
        return ret

In [2]:
class Polar:
    r: float = 0
    theta: float = 0

    def __init__(self):
        self.r = 0
        self.theta = 0

    def __init__(self, r, theta):
        self.r = r
        self.theta = theta

    def __str__(self):
        return self.r.__str__() + "e^(" + self.theta.__str__() + "i)"

    def __eq__(self, other: Polar) -> bool:
        return self.r == other.r and self.theta == other.theta

    def toCartesian(self) -> Complex:
        x = self.r * cos(self.theta)
        y = self.r * sin(self.theta)
        return Complex(x, y)

In [3]:
a = Complex(1, 2)
b = Complex(3, 4)
c = a + b
print(c)
c = a - b
print(c)
c = a * b
print(c)
c = a / b
print(c)

print(a)
print(a.modulus())

print(a.conjugate())

print(a.toPolar())
print(Polar(2, pi / 2))
a = Complex(1, -1)
print(a.pow(5))
a = Complex(3, -4)
print(a.toPolar())

4 + 6i
-2 + -2i
-5 + 10i
0.44 + 0.08i
1 + 2i
2.23606797749979
1 + -2i
2.23606797749979e^(1.1071487177940904i)
2e^(1.5707963267948966i)
-4.0 + 4.0i
5.0e^(-0.9272952180016122i)


In [4]:
class ComplexVector(list):
    def __init__(self, input):
        if isinstance(input, int):
            self.v = [Complex(0, 0) for _ in range(input)]
            self.n = input
        elif isinstance(input, list):
            self.v = input
            self.n = len(input)

    def __setitem__(self, index, item):
        self.v.__setitem__(index, item)

    def insert(self, index, item):
        self.v.insert(index, item)

    def append(self, item):
        self.v.append(item)

    def extend(self, other):
        self.v.extend(other)

    def __add__(self, other: ComplexVector) -> ComplexVector:
        ret = ComplexVector(self.n)
        for i in range(len(self.v)):
            ret.v[i] = self.v[i] + other.v[i]
        return ret

    def inverse(self) -> ComplexVector:
        ret = ComplexVector(self.n)
        for i in range(len(self.v)):
            ret.v[i] = self.v[i].negate()
        return ret

    def __mul__(self, c: Complex) -> ComplexVector:
        ret = ComplexVector(self.n)
        for i in range(len(self.v)):
            ret.v[i] = self.v[i] * c
        return ret

    def __eq__(self, other: ComplexVector) -> bool:
        for i in range(len(self.v)):
            if self.v[i] != other.v[i]:
                return False
        return True

    def mulByNorm(self, other: float) -> ComplexVector:
        complex = Complex(other, 0)
        return self * complex

    def __str__(self) -> str:
        finalString = ""
        for i in range(len(self.v)):
            finalString += "|" + self.v[i].__str__() + "|\n"
        return finalString

    def scalarProduct(self, other: ComplexVector) -> Complex:
        if self.n != other.n:
            raise Exception("Vectors must be of same length")
        ret = Complex(0, 0)
        for i in range(len(self.v)):
            ret += self.v[i] * other.v[i].conjugate()
        return ret

    def norm(self) -> float:
        return sqrt(self.scalarProduct(self).x)

    def distance(self, other: ComplexVector) -> float:
        return (self - other).norm()

    def tensorProduct(self, other: ComplexVector) -> ComplexVector:
        ret = ComplexVector(self.n * other.n)
        for i in range(len(self.v)):
            for j in range(len(other.v)):
                ret.v[i * len(other.v) + j] = self.v[i] * other.v[j]
        return ret

In [5]:
test = ComplexVector([
    Complex(1, 2),
    Complex(3, 4),
    Complex(5, 6),
    Complex(7, 8),
    Complex(0, 0)])
print(test + test)
print(test.inverse())
print(test * Complex(2, 0))
print(test.scalarProduct(test))
print(test.norm())

|2 + 4i|
|6 + 8i|
|10 + 12i|
|14 + 16i|
|0	|

|-1 + -2i|
|-3 + -4i|
|-5 + -6i|
|-7 + -8i|
|0	|

|2 + 4i|
|6 + 8i|
|10 + 12i|
|14 + 16i|
|0	|

204	
14.2828568570857


In [6]:
class ComplexMatrix(list):
    def __init__(self, n, m: int = 0):
        if isinstance(n, int) and isinstance(m, int):
            self.matrix = [ComplexVector(m) for _ in range(n)]
            self.n = n
            self.m = m
        elif isinstance(n, list) and isinstance(n[0], list):
            self.matrix = [ComplexVector(n[i]) for i in range(len(n))]  
            for i in range(len(n)):
                self.matrix[i] = ComplexVector(n[i])
            self.n = len(n)
            self.m = len(n[0])

    def __setitem__(self, index, item: ComplexVector):
        self.matrix.__setitem__(index, item)

    def insert(self, index, item:ComplexVector):
        self.matrix.insert(index, item)

    def append(self, item: ComplexVector):
        self.matrix.append(item)

    def extend(self, other: [ComplexVector]):
        self.matrix.extend(other)  

    def __add__(self, other: ComplexMatrix) -> ComplexMatrix:
        ret = ComplexMatrix(self.n, self.m)
        for i in range(self.n):
            ret.matrix[i] = self.matrix[i] + other.matrix[i]
        return ret

    def inverse(self) -> ComplexMatrix:
        ret = ComplexMatrix(self.n, self.m)
        for i in range(self.n):
            ret.matrix[i] = self.matrix[i].negate()
        return ret

    def Scalar(self, c: Complex) -> ComplexMatrix:
        ret = ComplexMatrix(self.n, self.m)
        for i in range(self.n):
            ret.matrix[i] = self.matrix[i] * c
        return ret

    def __str__(self) -> str:
        # print(np.matrix([self.matrix[i] for i in range(self.n)]).view())
        ret = ""
        for i in range(self.n):
            ret += "|"
            for j in range(self.m):
                ret += self.matrix[i].v[j].__str__()+ ", "
            ret = ret[:-2]
            ret += "|"
            ret += "\n"
        return ret

    def __mul__(self, other: ComplexMatrix) -> ComplexMatrix:
        if self.m != other.n:
            raise Exception("Invalid matrix multiplication")
        ret = ComplexMatrix(self.n, other.m)
        for i in range(self.n):
            for j in range(other.m):
                for k in range(self.m):
                    ret.matrix[i].v[j] += self.matrix[i].v[k] * other.matrix[k].v[j]
        return ret

    def transpose(self) -> ComplexMatrix:
        ret = ComplexMatrix(self.m, self.n)
        for i in range(self.n):
            for j in range(self.m):
                ret.matrix[j].v[i] = self.matrix[i].v[j]
        return ret

    def conjugate(self) -> ComplexMatrix:
        ret = ComplexMatrix(self.n, self.m)
        for i in range(self.n):
            for j in range(self.m):
                ret.matrix[i].v[j] = self.matrix[i].v[j].conjugate()
        return ret

    def dagger(self) -> ComplexMatrix:
        return self.transpose().conjugate()

    def multiplyByVector(self, vector: ComplexVector) -> ComplexVector:
        if self.m != vector.n:
            raise Exception("Invalid matrix multiplication")
        ret = ComplexVector(self.n)
        for i in range(self.n):
            for j in range(self.m):
                ret.v[i] += self.matrix[i].v[j] * vector.v[j]
        return ret

    @staticmethod
    def identity(n: int) -> ComplexMatrix:
        ret = ComplexMatrix(n, n)
        for i in range(n):
            ret.matrix[i].v[i] = Complex(1, 0)
        return ret

    @staticmethod
    def hadamardMatrix(M: int):
        n = 2**M
        # Initializing a matrix of order n
        hadamard = [[0] * n for _ in range(n)]

        # Initializing the 0th column and
        # 0th row element as 1
        hadamard[0][0] = 1

        k = 1
        while k < n:
            # Loop to copy elements to
            # other quarters of the matrix
            for i in range(k):
                for j in range(k):
                    hadamard[i + k][j] = hadamard[i][j]
                    hadamard[i][j + k] = hadamard[i][j]
                    hadamard[i + k][j + k] = -hadamard[i][j]
            k *= 2

        # Displaying the final hadamard matrix
        for i in range(n):
            for j in range(n):
                print(hadamard[i][j], end=" ")
            print()

    def __eq__(self, other: ComplexMatrix) -> bool:
        for i in range(self.n):
            if self.matrix[i] != other.matrix[i]:
                return False
        return True

    def isHermitian(self) -> bool:
        return self == self.dagger()

    def isUnitary(self) -> bool:
        leftSide = self * self.dagger()
        rightSide = self.dagger() * self
        return leftSide == rightSide == ComplexMatrix.identity(self.n)

    def tensorProduct(self, other: ComplexMatrix) -> ComplexMatrix:
        ret = ComplexMatrix(self.n * other.n, self.m * other.m)
        for i in range(self.n):
            for j in range(self.m):
                for k in range(other.n):
                    for l in range(other.m):
                        ret.matrix[i * other.n + k].v[j * other.m + l] = (
                            self.matrix[i].v[j] * other.matrix[k].v[l]
                        )
        return ret

In [7]:
testMatrix = ComplexMatrix( [
    [ Complex(1, 2), Complex(3, 4) ],
    [ Complex(5, 6), Complex(7, 8) ]])
print(testMatrix)
print(testMatrix +testMatrix)
# create 2 test matrices for multiplication
testMatrix1 = ComplexMatrix([
    [Complex(3,2), Complex(0,0), Complex(5,-6)],
    [Complex(1,0), Complex(4,2), Complex(0,1)],
    [Complex(4,-1), Complex(0,0), Complex(4,0)]])
print (testMatrix1)
testMatrix2 = ComplexMatrix([
    [Complex(5,0), Complex(2,-1), Complex(6,-4)],
    [Complex(0,0), Complex(4,5), Complex(2,0)],
    [Complex(7,-4), Complex(2,7), Complex(0,0)]])

print(testMatrix1 * testMatrix2)
print (testMatrix1.Scalar(Complex(2, 0)))
print(testMatrix1.transpose())
print(testMatrix1.conjugate())
print(testMatrix1.dagger())

testVector = ComplexVector([
    Complex(1, 2),
    Complex(3, 4),
    Complex(5, 6)])
print(testMatrix1.multiplyByVector(testVector))
print (ComplexMatrix.hadamardMatrix(1))
print (ComplexMatrix.hadamardMatrix(3))

hermitianMatrix = ComplexMatrix([
    [ Complex(7, 0), Complex(6, 5)],
    [ Complex(6, -5), Complex(-3, 0)]])
print(hermitianMatrix.isHermitian())
print(hermitianMatrix.isUnitary())
theta = pi/4
unitaryMatrix = ComplexMatrix([
    [Complex(cos(theta), 0), Complex(0, -sin(theta)), Complex(0, 0)],
    [Complex(0, -sin(theta)), Complex(cos(theta), 0), Complex(0, 0)],
    [Complex(0, 0), Complex(0, 0), Complex(1, 0)]])

print(unitaryMatrix.isHermitian())
print(unitaryMatrix.isUnitary())

# tensor product
tensorProductMatrix = ComplexMatrix([
    [Complex(1,0), Complex(2,0)],
    [Complex(3,0), Complex(4,0)]])
print(tensorProductMatrix.tensorProduct(tensorProductMatrix))

|1 + 2i 3 + 4i |
|5 + 6i 7 + 8i |

|2 + 4i 6 + 8i |
|10 + 12i 14 + 16i |

|3 + 2i 0	 5 + -6i |
|1	 4 + 2i 1i	 |
|4 + -1i 0	 4	 |

|26 + -52i 60 + 24i 26	 |
|9 + 7i 1 + 29i 14	 |
|48 + -21i 15 + 22i 20 + -22i |

|6 + 4i 0	 10 + -12i |
|2	 8 + 4i 2i	 |
|8 + -2i 0	 8	 |

|3 + 2i 1	 4 + -1i |
|0	 4 + 2i 0	 |
|5 + -6i 1i	 4	 |

|3 + -2i 0	 5 + 6i |
|1	 4 + -2i -1i	 |
|4 + 1i 0	 4	 |

|3 + -2i 1	 4 + 1i |
|0	 4 + -2i 0	 |
|5 + 6i -1i	 4	 |

|60 + 8i|
|-1 + 29i|
|26 + 31i|

1 1 
1 -1 
None
1 1 1 1 1 1 1 1 
1 -1 1 -1 1 -1 1 -1 
1 1 -1 -1 1 1 -1 -1 
1 -1 -1 1 1 -1 -1 1 
1 1 1 1 -1 -1 -1 -1 
1 -1 1 -1 -1 1 -1 1 
1 1 -1 -1 -1 -1 1 1 
1 -1 -1 1 -1 1 1 -1 
None
True
True
True
True
|1	 2	 2	 4	 |
|3	 4	 6	 8	 |
|3	 6	 4	 8	 |
|9	 12	 12	 16	 |



In [8]:
def matrixToThePower(matrix: [[bool]], power:int) -> [[bool]]:
    newMatrix = matrix
    for _ in range(power-1):
        newMatrix =boolMatrixMultiplication(newMatrix, matrix)
    return newMatrix

def boolMatrixMultiplication(matrix1: [[bool]], matrix2: [[bool]]) -> [[bool]]:
    if len(matrix1[0]) != len(matrix2):
        raise Exception("Invalid matrix multiplication")
    ret = [[False for _ in range(len(matrix2[0]))] for _ in range(len(matrix1))]
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            for k in range(len(matrix2)):
                ret[i][j] = ret[i][j] or (matrix1[i][k] and matrix2[k][j])
    return ret

def marbleExperiment(matrix: [[bool]], marblesInTheStates: [int], iterations: int)-> [int]:
    if len(matrix) != len(marblesInTheStates):
        raise Exception("Invalid input")
    newMatrix = matrixToThePower(matrix, iterations)
    newMarbles = [0 for _ in range(len(marblesInTheStates))]
    for i in range(len(marblesInTheStates)):
        for j in range(len(marblesInTheStates)):
            if newMatrix[j][i]:
                newMarbles[j] += marblesInTheStates[i]
    return newMarbles

In [9]:
connections = [ [False, False, False, False, False, False],
                [False, False, False, False, False, False],
                [False, True,  False, False, False, True ],
                [False, False, False, True,  False, False],
                [False, False, True,  False, False, False],
                [True, False, False, False, True,  False]]
marbles = [6,2,1,5,3,10]
newMarbles = marbleExperiment(connections, marbles,1)
print(newMarbles)

[0, 0, 12, 5, 1, 9]


In [10]:
def matrixToThePowerFloat(matrix: [[float]], power:int) -> [[float]]:
    newMatrix = matrix
    for _ in range(power-1):
        newMatrix =floatMatrixMultiplication(newMatrix, matrix)
    return newMatrix

def floatMatrixMultiplication(matrix1: [[float]], matrix2: [[float]]) -> [[float]]:
    if len(matrix1[0]) != len(matrix2):
        raise Exception("Invalid matrix multiplication")
    ret = [[0 for _ in range(len(matrix2[0]))] for _ in range(len(matrix1))]
    for i in range(len(matrix1)):
        for j in range(len(matrix2[0])):
            for k in range(len(matrix2)):
                ret[i][j] += matrix1[i][k] * matrix2[k][j]
    return ret

def fancyPrint(matrix: [[float]]):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            print(format(matrix[i][j],".2f"), end="\t")
        print()

def marbleExperimentFloat(matrix: [[float]], marblesInTheStates: [int], iterations: int)-> [float]:
    if len(matrix) != len(marblesInTheStates):
        raise Exception("Invalid input")
    newMatrix = matrixToThePowerFloat(matrix, iterations)
    newMarbles = [float(0) for _ in range(len(marblesInTheStates))]
    for i in range(len(marblesInTheStates)):
        for j in range(len(marblesInTheStates)):
            newMarbles[j] += newMatrix[j][i] * marblesInTheStates[i]
    return newMarbles

In [11]:
billiardBall = [[0, 0.5, 0.5, 0],
                [0.5, 0, 0, 0.5],
                [0.5, 0, 0, 0.5],
                [0, 0.5, 0.5, 0]]
marbles = [1,0,0,0]
print(marbleExperimentFloat(billiardBall, marbles, 3))

[0.0, 0.5, 0.5, 0.0]


In [12]:
def slitExperiment(numberOfSlits:int, numberOfTargets:int, slitProbabilities:[float], targetProbabilities:[[float]]):
    constructFloatMatrix: [[float]] =  [[0 for _ in range(numberOfSlits + numberOfTargets+1)] for _ in range(numberOfSlits + numberOfTargets+1)]
    for i in range(numberOfSlits):
        constructFloatMatrix[i+1][0] = slitProbabilities[i]
    for i in range(numberOfSlits):
        for j in range(numberOfTargets):
            constructFloatMatrix[numberOfSlits+j+1][i+1] = targetProbabilities[i][j]
    for i in range(numberOfTargets):
        constructFloatMatrix[numberOfSlits+i+1][numberOfSlits+1+i] = 1
    fancyPrint(constructFloatMatrix)

    initialMarbles = [0 for _ in range(numberOfSlits + numberOfTargets+1)]
    initialMarbles[0] = 1
    print(initialMarbles)
    print(marbleExperimentFloat(constructFloatMatrix, initialMarbles, 2))



In [13]:
# slitExperiment(2, 4, [0.5, 0.5], [[0.5, 0.5,0,0], [0,0,0.5, 0.5]])
slitExperiment(2, 5, [0.5,0.5],[[1/3, 1/3, 1/3,0,0], [0,0,1/3,1/3, 1/3]])
slitExperiment(3, 5, [1/3,1/3,1/3],[[1/3, 1/3, 1/3,0,0], [0,1/3,1/3, 1/3,0], [0,0,1/3, 1/3, 1/3]])

0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.50	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.50	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.00	0.33	0.00	1.00	0.00	0.00	0.00	0.00	
0.00	0.33	0.00	0.00	1.00	0.00	0.00	0.00	
0.00	0.33	0.33	0.00	0.00	1.00	0.00	0.00	
0.00	0.00	0.33	0.00	0.00	0.00	1.00	0.00	
0.00	0.00	0.33	0.00	0.00	0.00	0.00	1.00	
[1, 0, 0, 0, 0, 0, 0, 0]
[0.0, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666]
0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.33	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.33	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.33	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.00	
0.00	0.33	0.00	0.00	1.00	0.00	0.00	0.00	0.00	
0.00	0.33	0.33	0.00	0.00	1.00	0.00	0.00	0.00	
0.00	0.33	0.33	0.33	0.00	0.00	1.00	0.00	0.00	
0.00	0.00	0.33	0.33	0.00	0.00	0.00	1.00	0.00	
0.00	0.00	0.00	0.33	0.00	0.00	0.00	0.00	1.00	
[1, 0, 0, 0, 0, 0, 0, 0, 0]
[0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.2222222222222222, 0.3333333333333333, 0.22

In [14]:
import copy
def matrixToThePowerComplex(matrix: ComplexMatrix, power:int) -> ComplexMatrix:
    newMatrix = copy.deepcopy(matrix)
    for _ in range(power-1):
        newMatrix = newMatrix * matrix
    return newMatrix

def marbleExperimentComplex(matrix: ComplexMatrix, marblesInTheStates: ComplexVector, iterations: int)-> ComplexVector:
    if matrix.n != marblesInTheStates.n:
        raise Exception("Invalid input")
    newMatrix = matrixToThePowerComplex(matrix, iterations)
    return newMatrix.multiplyByVector(marblesInTheStates)

In [15]:
quantumBilliard  = ComplexMatrix([
    [Complex(0,0), Complex(1/sqrt(2),0), Complex(1/sqrt(2),0), Complex(0,0)],
    [Complex(1/sqrt(2),0), Complex(0,0), Complex(0,0), Complex(-1/sqrt(2),0)],
    [Complex(1/sqrt(2),0), Complex(0,0), Complex(0,0), Complex(1/sqrt(2),0)],
    [Complex(0,0), Complex(-1/sqrt(2),0), Complex(1/sqrt(2),0), Complex(0,0)]])
print(quantumBilliard)

initialMarbles = ComplexVector(4)
initialMarbles.v[0] = Complex(1, 0)

print(marbleExperimentComplex(quantumBilliard, initialMarbles, 1))
print()
print(marbleExperimentComplex(quantumBilliard, initialMarbles, 2))
print()
print(marbleExperimentComplex(quantumBilliard, initialMarbles, 3))
print()
print(marbleExperimentComplex(quantumBilliard, initialMarbles, 4))

|0	 0.71	 0.71	 0	 |
|0.71	 0	 0	 -0.71	 |
|0.71	 0	 0	 0.71	 |
|0	 -0.71	 0.71	 0	 |

|0.0	|
|0.71	|
|0.71	|
|0.0	|


|1.0	|
|0.0	|
|0.0	|
|0.0	|


|0.0	|
|0.71	|
|0.71	|
|0.0	|


|1.0	|
|0.0	|
|0.0	|
|0.0	|



In [16]:
def slitExperimentComplex(numberOfSlits:int, numberOfTargets:int, slitProbabilities:ComplexVector, targetProbabilities:[ComplexVector]):
    complexMatrix = ComplexMatrix(numberOfSlits + numberOfTargets+1, numberOfSlits + numberOfTargets+1)

    for i in range(numberOfSlits):
        complexMatrix.matrix[i+1].v[0] = slitProbabilities.v[i]
    for i in range(numberOfSlits):
        for j in range(numberOfTargets):
            complexMatrix.matrix[numberOfSlits+j+1].v[i+1] = targetProbabilities[i].v[j]
    for i in range(numberOfTargets):
        complexMatrix.matrix[numberOfSlits+i+1].v[numberOfSlits+1+i] = Complex(1,0)
    print(complexMatrix)

    initialMarbles = ComplexVector(numberOfSlits + numberOfTargets+1)
    initialMarbles.v[0] = Complex(1, 0)
    print(initialMarbles)

    print(marbleExperimentComplex(complexMatrix, initialMarbles, 2))



In [17]:
slitProbabilities = ComplexVector([
    Complex(1/sqrt(2), 0),
    Complex(1/sqrt(2), 0)])

targetProbabilities = [ComplexVector(5) for _ in range(2)]
targetProbabilities[0].v[0] = Complex(-1/sqrt(6),1/sqrt(6))
targetProbabilities[0].v[1] = Complex(-1/sqrt(6),-1/sqrt(6))
targetProbabilities[0].v[2] = Complex(1/sqrt(6),-1/sqrt(6))
targetProbabilities[1].v[2] = Complex(-1/sqrt(6),1/sqrt(6))
targetProbabilities[1].v[3] = Complex(-1/sqrt(6),-1/sqrt(6))
targetProbabilities[1].v[4] = Complex(1/sqrt(6),-1/sqrt(6))

print(slitExperimentComplex(2, 5, slitProbabilities, targetProbabilities))

|0	 0	 0	 0	 0	 0	 0	 0	 |
|0.71	 0	 0	 0	 0	 0	 0	 0	 |
|0.71	 0	 0	 0	 0	 0	 0	 0	 |
|0	 -0.41 + 0.41i 0	 1	 0	 0	 0	 0	 |
|0	 -0.41 + -0.41i 0	 0	 1	 0	 0	 0	 |
|0	 0.41 + -0.41i -0.41 + 0.41i 0	 0	 1	 0	 0	 |
|0	 0	 -0.41 + -0.41i 0	 0	 0	 1	 0	 |
|0	 0	 0.41 + -0.41i 0	 0	 0	 0	 1	 |

|1	|
|0	|
|0	|
|0	|
|0	|
|0	|
|0	|
|0	|

|0.0	|
|0.0	|
|0.0	|
|-0.29 + 0.29i|
|-0.29 + -0.29i|
|0.0	|
|-0.29 + -0.29i|
|0.29 + -0.29i|

None


In [18]:
def quantumSystem(numberOfPoint: int, initialStates:ComplexVector,endStates:ComplexVector = ComplexVector(0)): # initialStates -> amplitudes
    normOfStates = initialStates.norm()
    print(initialStates)
    for i in range(numberOfPoint):
        print ("Probability of being at point " + i.__str__() + " is " + (initialStates.v[i].modulus() ** 2 / normOfStates ** 2).__str__())

    #normalized
    normalizedStates = initialStates.mulByNorm(1/normOfStates)
    print(normalizedStates)
    print(normalizedStates.norm())

    for i in range(numberOfPoint):
        print ("Probability of being at point " + i.__str__() + " is " + (normalizedStates.v[i].modulus() ** 2).__str__())

    if endStates.n != 0:
        normalizedEndStates = endStates.mulByNorm(1/endStates.norm())
        resultAmplitude = normalizedEndStates.scalarProduct(normalizedStates)
        print("Probability of being at end state is " + (resultAmplitude.modulus() ** 2).__str__())


    

In [19]:
inputStates = ComplexVector( 
    [Complex(-3,-1),
     Complex(0,-2),
     Complex(0,1),
     Complex(2,0)]);
endStates = ComplexVector(
    [Complex(-3,-1),
     Complex(0,-2),
     Complex(0,1),
     Complex(2,0)]);
quantumSystem(4, inputStates,endStates)


|-3 + -1i|
|-2i	|
|1i	|
|2	|

Probability of being at point 0 is 0.5263157894736842
Probability of being at point 1 is 0.21052631578947364
Probability of being at point 2 is 0.05263157894736841
Probability of being at point 3 is 0.21052631578947364
|-0.69 + -0.23i|
|-0.46i	|
|0.23i	|
|0.46	|

0.9999999999999998
Probability of being at point 0 is 0.526315789473684
Probability of being at point 1 is 0.21052631578947364
Probability of being at point 2 is 0.05263157894736841
Probability of being at point 3 is 0.21052631578947364
Probability of being at end state is 0.9999999999999993
